<a href="https://colab.research.google.com/github/subspecs/Cocaine/blob/master/notebooks/piper_model_exporter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color="ffc800"> **[Piper](https://github.com/rhasspy/piper) model exporter.**
## ![Piper logo](https://contribute.rhasspy.org/img/logo.png)
---

* Notebook created by: [rmcpantoja](http://github.com/rmcpantoja)
* Collaborator: [Xx_Nessu_xX](http://github.com/XxNessuxX)

In [4]:
#@markdown # <font color="ffc800"> **Install software.** 📦
#@markdown ---

print("\033[93mInstalling...")
!git clone -q https://github.com/rhasspy/piper
%cd /content/piper/src/python
!pip install pip==24.0
!pip install -q cython>=0.29.0 librosa>=0.9.2 numpy>=1.19.0 pytorch-lightning~=1.9.0 torch~=2.3.0
!pip install -q onnx onnxruntime-gpu
!bash build_monotonic_align.sh
!apt-get install espeak-ng
!pip install -q torchtext==0.15.2
!pip install -q torchaudio==2.0.2 torchmetrics==0.11.4
!pip install torch -U
#!pip install --upgrade gdown

print("\033[93mDone!")

Installing...
/content/piper/src/python
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2 requires torch==2.0.1, but you have torch 2.3.1 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 2.3.1 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 2.3.1 which is incompatible.
torchvision 0.20.1+cu121 requires torch==2.5.1, but you have torch 2.3.1 which is incompatible.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
espeak-ng is already the newest version (1.50+dfsg-10ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.20.1+cu121 

In [5]:
#@markdown # <font color="ffc800"> **Voice package generation section.** 🗣️
#@markdown ---
%cd /content/piper/src/python
import os
import json
import ipywidgets as widgets
from IPython.display import display
import json
from google.colab import output
guideurl = "https://github.com/rmcpantoja/piper/blob/master/notebooks/wav/en"
#@markdown #### *Download:*
#@markdown **Drive ID or direct download link of the model in another cloud:**
model_id = "https://drive.google.com/file/d/1-7vll2NWNBqoliglZvFjdQ6cEaOEoxax/view?usp=sharing" #@param {type:"string"}
#@markdown **Drive ID or direct download link of the config.json file:**
config_id = "https://drive.google.com/file/d/1-CE84QOoR6haOAxEyKNz-tFO9dkTLzJP/view?usp=sharing" #@param {type:"string"}
#@markdown ---

#@markdown #### *Creation process:*
#@markdown **Choose the language code (iso639-1 format):**

#@markdown You can see a list of language codes and names [here](https://www.loc.gov/standards/iso639-2/php/English_list.php).

language = "en_US" #@param ["ar_JO", "ca_ES", "cs_CZ", "da_DK", "de_DE", "el_GR", "en_GB", "en_US", "es_ES", "es_LA", "fi_FI", "fr_FR", "grc", "hu_GU", "is_IS", "it_IT", "kk_KZ", "ka_GE", "lb_LU", "nb", "ne", "nl_BE", "no_NO", "pl_PL", "pt_BR", "pt_PT", "ro_RO", "ru_RU", "sk_SK", "sr", "sv_SE", "sw_CD", "tr_TR", "uk_UA", "vi_VN", "zh_CN"]
voice_name = "Yui" #@param {type:"string"}
voice_name = voice_name.lower()
quality = "medium" #@param ["high", "low", "medium", "x-low"]
#@markdown **Do you want to write a model card?** *(Optional.)*
write_model_card = False #@param {type:"boolean"}

#@markdown **Do you want this voice to have a faster response speed?**
streaming = False #@param {type:"boolean"}

def start_process(streaming):
    if not os.path.exists("/content/project/model.ckpt"):
        raise Exception("Could not download model! make sure the file is shareable to everyone")
    output.eval_js(f'new Audio("{guideurl}/starting.wav?raw=true").play()')
    if not streaming:
        !python -m piper_train.export_onnx "/content/project/model.ckpt" "{export_voice_path}/{export_voice_name}.onnx"
    else:
        !python -m piper_train.export_onnx_streaming "/content/project/model.ckpt" "{export_voice_path}"
    print("\033[93mCompressing...")
    !tar -czvf "{packages_path}/{export_voice_name}.tar.gz" -C "{export_voice_path}" .
    output.eval_js(f'new Audio("{guideurl}/success.wav?raw=true").play()')
    print("\033[93mDone!")

if not streaming:
    export_voice_name = f"{language}-{voice_name}-{quality}"
else:
    export_voice_name = f"{language}-{voice_name}+RT-{quality}"
export_voice_path = "/content/project/voice-"+export_voice_name
packages_path = "/content/project/packages"
if not os.path.exists(export_voice_path):
    os.makedirs(export_voice_path)
if not os.path.exists(packages_path):
    os.makedirs(packages_path)
print("\033[93mDownloading model and his config...")
if model_id.startswith("1"):
    !gdown -q "{model_id}" -O /content/project/model.ckpt
elif model_id.startswith("https://drive.google.com/file/d/"):
    !gdown -q "{model_id}" -O "/content/project/model.ckpt" --fuzzy
else:
    !wget "{model_id}" -O "/content/project/model.ckpt"
if config_id.startswith("1"):
    !gdown -q "{config_id}" -O "{export_voice_path}/{export_voice_name}.onnx.json"
elif config_id.startswith("https://drive.google.com/file/d/"):
    !gdown -q "{config_id}" -O "{export_voice_path}/{export_voice_name}.onnx.json" --fuzzy
else:
    !wget "{config_id}" -O "{export_voice_path}/{export_voice_name}.onnx.json"

if os.path.exists(f"{export_voice_path}/{export_voice_name}.onnx.json") and streaming:
    with open(f"{export_voice_path}/{export_voice_name}.onnx.json", "r", encoding="utf-8") as f:
        tmp = f.read()
    new_config = json.loads(tmp)
    new_config["streaming"] = True
    new_config["key"] = export_voice_name

    with open(f"{export_voice_path}/{export_voice_name}.onnx.json", "w", encoding="utf-8") as f_new:
        json.dump(new_config, f_new, indent=4)

if write_model_card:
    with open(f"{export_voice_path}/{export_voice_name}.onnx.json", "r") as file:
        config = json.load(file)
    sample_rate = config["audio"]["sample_rate"]
    num_speakers = config["num_speakers"]
    output.eval_js(f'new Audio("{guideurl}/waiting.wav?raw=true").play()')
    text_area = widgets.Textarea(
        description = "fill in this following template and press start to generate the voice package",
        value=f'# Model card for {voice_name} ({quality})\n\n* Language: {language} (normaliced)\n* Speakers: {num_speakers}\n* Quality: {quality}\n* Samplerate: {sample_rate}Hz\n\n## Dataset\n\n* URL: \n* License: \n\n## Training\n\nTrained from scratch.\nOr finetuned from: ',
        layout=widgets.Layout(width='500px', height='200px')
    )
    button = widgets.Button(description='Start')

    def create_model_card(button):
        model_card_text = text_area.value.strip()
        with open(f'{export_voice_path}/MODEL_CARD', 'w') as file:
            file.write(model_card_text)
        text_area.close()
        button.close()
        output.clear()
        start_process(streaming)

    button.on_click(create_model_card)

    display(text_area, button)
else:
    start_process(streaming)

/content/piper/src/python
2025-01-26 03:24:40.916699: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-26 03:24:41.313058: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-26 03:24:41.415224: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-26 03:24:42.050396: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-26 03:24:45.665085:

In [ ]:
#@markdown # <font color="ffc800"> **Download/export your generated voice package.** 📥
#@markdown ---

#@markdown #### *How do you want to export your model?*
export_mode = "upload it to my Google Drive" #@param ["Download the voice package on my device (may take some time)", "upload it to my Google Drive"]
print("\033[93mExporting package...")
if export_mode == "Download the voice package on my device (may take some time)":
    from google.colab import files
    files.download(f"{packages_path}/{export_voice_name}.tar.gz")
    msg = "Please wait a moment while the package is being downloaded."
else:
    voicepacks_folder = "/content/drive/MyDrive/piper voice packages"
    from google.colab import drive
    drive.mount('/content/drive')
    if not os.path.exists(voicepacks_folder):
        os.makedirs(voicepacks_folder)
    !cp "{packages_path}/{export_voice_name}.tar.gz" "{voicepacks_folder}"
    msg = f"You can find the generated voice package at: {voicepacks_folder}."
print(f"\033[93mDone! {msg}")

# "*I want to test this model! I don't need anything else anymore?*"

No, this is almost the end! Now you can share your generated package to your friends, upload to a cloud storage and/or test it on:
* [The inference notebook](https://colab.research.google.com/github/rmcpantoja/piper/blob/master/notebooks/piper_inference_(ONNX).ipynb)
  * run the cells in order for it to work correctly, as well as all the notebooks. Also, the inference notebook will guide you through the process using the enhanced accessibility feature if you wish. It's easy to use. Test it!
* Or through the NVDA screen reader!
  * Download and install the latest version of the [add-on](https://github.com/mush42/piper-nvda/releases).
  * Once the add-on is installed, go to NVDA menu/piper voice manager...
  * In the installed voices page, tab until you find the `Install from local file` button, press enter and select the generated package in your downloads.
  * Once the package is selected and installed, apply the changes and restart NVDA to update the voice list.
* Enjoy your creation!